# Uncertainty Engine SDK - save and load

This notebook goes through some of the basic functionality of the SDK. It demonstrates how to save and load workflows.


## The `Client`

Start by importing the `Client`.


In [1]:
from uncertainty_engine import Client, Environment
from pprint import pprint

Next initialise the `Client` with the Uncertainty Engine environment to target.

In [2]:
import os

client = Client(
    env=Environment(
        cognito_user_pool_client_id=os.environ["UE_COGNITO_CLIENT_ID"],
        core_api=os.environ["UE_CORE_API"],
        region=os.environ["UE_REGION"],
        resource_api=os.environ["UE_RESOURCE_API"],
    ),
)

The `Client` is an important concept in the SDK as all interactions with the Uncertainty Engine go through the `Client`. Before accessing your resources you must authenticate with your account ID.

In [3]:
# client.authenticate(os.environ["UE_ACCOUNT_ID"])
PROJECT_ID = os.environ["UE_PROJECT_ID"]

## Saving

In [4]:
from uncertainty_engine.graph import Graph
from uncertainty_engine.nodes.basic import Add

graph = Graph()

add = Add(
  rhs=1,
  lhs=42,
  label="Add"
)

# Add the node to the graph
graph.add_node(add)

# Define our add output handle
ans = add.make_handle("ans")

In [5]:
from uncertainty_engine.nodes.workflow import Workflow

workflow = Workflow(
    graph=graph.nodes,
    input=graph.external_input,
    external_input_id=graph.external_input_id,
    requested_output={
      "Add": ans.model_dump()
    }
)

pprint(workflow.__dict__)

{'external_input_id': '_',
 'graph': {'nodes': {'Add': {'inputs': {'lhs': {'node_handle': 'Add_lhs',
                                                'node_name': '_'},
                                        'rhs': {'node_handle': 'Add_rhs',
                                                'node_name': '_'}},
                             'type': 'Add'}}},
 'inputs': {'Add_lhs': 42, 'Add_rhs': 1},
 'label': None,
 'node_name': 'Workflow',
 'requested_output': {'Add': {'node_handle': 'ans', 'node_name': 'Add'}}}


In [6]:
# workflow_id = client.workflows.save(
#     project_id=PROJECT_ID,
#     workflow=workflow,
#     workflow_name="NewNewWorkflow"
# )

Once authenticate you will have access to all of your available resources. In the Uncertainty Engine, resources always belong to a `project`, this helps keep your work organised and allows you to share resources with other users. You can think of a `project` as a folder that contains all of your resources. To make life easier we will define our project ID here.

Resources are split into different categories (e.g `dataset`, `document`, `model`, ...), listing resources is filtered by these categories. 

(We use `pprint` to make the data display a bit nicer)

In [7]:


my_workflows = client.workflows.list_workflows(PROJECT_ID)

pprint(my_workflows)

[{'created_at': '14:59:35 2025-05-15',
  'id': '68260157982da250adee2828',
  'name': 'Chatwithcontext',
  'versions': ['68260158982da250adee2829']},
 {'created_at': '14:59:48 2025-05-15',
  'id': '68260164982da250adee282a',
  'name': 'Chatsimple',
  'versions': ['68260164982da250adee282b']},
 {'created_at': '16:17:59 2025-06-11',
  'id': '6849ac37d465c24bf9e9e34d',
  'name': 'no-version-wf',
  'versions': []},
 {'created_at': '10:06:09 2025-06-12',
  'id': '684aa691d465c24bf9e9e356',
  'name': 'FunWorkflow',
  'versions': ['684aa691d465c24bf9e9e357',
               '684aa776d465c24bf9e9e358',
               '684aa7cbd465c24bf9e9e359',
               '684aa8b2d465c24bf9e9e35a',
               '684aa9b9d465c24bf9e9e35b',
               '684ab0c5d465c24bf9e9e35c',
               '684ab13cd465c24bf9e9e35d',
               '684acd90d465c24bf9e9e362']},
 {'created_at': '12:53:15 2025-06-12',
  'id': '684acdbbd465c24bf9e9e363',
  'name': 'NewWorkflow',
  'versions': ['684acdbbd465c24bf9e9e364

## Uploading Resources

When uploading a resource, it must be to a project (using the project ID) and you must provide a name and resource type. The name is the user friendly display name of the resource while the resource ID is the value used to retrieve the resource later. The resource ID is automatically generated by the Uncertainty Engine and is unique to the project.

In [8]:
fun_workflow = client.workflows.load(
    project_id=PROJECT_ID,
    workflow_id="684aa691d465c24bf9e9e356",
    version_id="684ab0c5d465c24bf9e9e35c"
)

pprint(fun_workflow.__dict__)

{'external_input_id': '_',
 'graph': {'nodes': {'Add': {'inputs': {'lhs': {'node_handle': 'Add_lhs',
                                                'node_name': '_'},
                                        'rhs': {'node_handle': 'Add_rhs',
                                                'node_name': '_'}},
                             'type': 'Add'}}},
 'inputs': {'Add_lhs': 2, 'Add_rhs': 1},
 'label': None,
 'node_name': 'Workflow',
 'requested_output': {'Add': {'node_handle': 'ans', 'node_name': 'Add'}}}


In [9]:
response = client.run_node(fun_workflow)
pprint(response)

{'inputs': {'external_input_id': '_',
            'graph': {'nodes': {'Add': {'inputs': {'lhs': {'node_handle': 'Add_lhs',
                                                           'node_name': '_'},
                                                   'rhs': {'node_handle': 'Add_rhs',
                                                           'node_name': '_'}},
                                        'type': 'Add'}}},
            'inputs': {'Add_lhs': 2, 'Add_rhs': 1},
            'requested_output': {'Add': {'node_handle': 'ans',
                                         'node_name': 'Add'}}},
 'message': 'Job completed at 2025-06-12 12:56:29.422283',
 'outputs': {'outputs': {'Add': 3.0}},
 'progress': {'Add': {'inputs': {'lhs': 2, 'rhs': 1},
                      'message': 'Job completed at 2025-06-12 12:56:29.009693',
                      'outputs': {'ans': 3.0},
                      'progress': None,
                      'status': 'completed'}},
 'status': 'completed'}
